In [ ]:
import pandas as pd
import tabula

In [ ]:
class Person:
    def __init__(self, nome, mensalidade):
        self.nome = nome
        self.mensalidade = mensalidade
        self.dependencia = None
        self.data_nascimento = None
        self.data_inclusao = None
        self.dependentes = []
        self.utilizacao = 0
        self.taxa_boleto = 2.93
        self.reaj_retro = 0
        self.reaj_faixa = 0
        
    def get_valor_mensalidades(self):
        valor_dependentes = sum([x.mensalidade for x in self.dependentes])
        return self.mensalidade + valor_dependentes
    
    def get_valor_utilizacao(self):
        valor_dependentes = sum(x.utilizacao for x in self.dependentes)
        return self.utilizacao + valor_dependentes
    
    def get_valor_reaj_retroativo(self):
        valor_dependentes = sum([x.reaj_retro for x in self.dependentes])
        return self.reaj_retro + valor_dependentes
    
    def get_valor_reaj_faixa(self):
        valor_dependentes = sum([x.reaj_faixa for x in self.dependentes])
        return self.reaj_faixa + valor_dependentes
    
    def get_valor_total(self):
        return self.get_valor_mensalidades() + self.get_valor_utilizacao() + self.taxa_boleto + self.get_valor_reaj_retroativo() + self.get_valor_reaj_faixa()    
    
    def __str__(self):
        return f'{self.nome} - {self.get_valor_total()}'
    
    def __repr__(self):
        return f'{self.nome} - {self.get_valor_total()}'
    
    def report_values(self):
        l = []
        l.append(f'{self.nome}: {self.mensalidade:.2f}')
        l.append(f'Utilização: {self.utilizacao:.2f}')
        l.append(f'Reajuste retroativo: {self.reaj_retro:.2f}')
        l.append(f'Reajuste faixa etária: {self.reaj_faixa:.2f}')
        for p in self.dependentes:
            l.append(f'{p.nome}: {p.mensalidade:.2f}')
            l.append(f'Reajuste retroativo: {p.reaj_retro:.2f}')
            l.append(f'Reajuste faixa etária: {p.reaj_faixa:.2f}')
        return l
    
    

In [ ]:
p = Person('Juliano Fischer Naves',400)
p1 = Person('Jhaneffer',300)
p2 = Person('Juliane',200)
p3 = Person('Olívia',100)
l = [p1, p2, p3]
p.dependentes.extend(l)

In [ ]:
p.get_valor_total()

In [ ]:
p.mensalidade

In [ ]:
FILE_REPORT_CONTRATOS = "faturamentoagosto21/Relatório de Mensalidades 67.pdf"
FILE_REPORT_USO = "faturamentoagosto21/Relatório de Utilização 67.pdf"

In [ ]:
df = tabula.read_pdf(FILE_REPORT_CONTRATOS, pages='all')

In [ ]:
df[0].head()

In [ ]:
df[13]

In [ ]:
#df[13]
df[13].drop([9,10], inplace=True)
#df[13]
#df[8] tá zikado
#df[1].drop('C/Corrente',axis=1)

In [ ]:
df[13]

In [ ]:
def df_filter_procedure(df):
    #df = df.drop(['Unnamed: 7', 'Unnamed: 2'], axis=1)
    #df = df.drop(df.columns[-1],axis=1)
    df = df.drop(['Plano', 'Total', 'C/Corrente','Lotação'], axis=1)
    df = df.dropna()
    #df.drop(df.index[0], inplace=True)
    values = ['contrato', 'nome', 'dependencia', 'data_nascimento', 'data_inclusao', 'mensalidade', 'reajuste_retroativo', 'reajuste_faixa']
    keys = ['Beneficiário', 'Nome', 'Dependência', 'Dt. Nascimento', 'Dt. Inclusão', 'Mensalidade','Reaj.\rRetroativo','Reaj.\rFaixa']
    d = dict(zip(keys, values))
    #df['Unnamed: 1'] = df['Unnamed: 1'].apply(lambda x: x.split(" Nac")[0])
    # df = df.drop(['nome','plano'],axis=1)
    df.rename(columns=d, inplace=True)
    return df

In [ ]:
pages = []
#ignora a última porque o df[8] tá zikado
#for page in df[:-1]:
for page in df:
    pages.append(page)
pages = pd.concat(pages)

In [ ]:
#até aqui tudo bem... arrumar um jeito de filtrar o df[8]
#df[8]

In [ ]:
#descarta as duas primeiras linhas e a última
'''df[8].drop([0,1,19], inplace=True)
df[8].drop(['Unnamed: 7', 'Unnamed: 4'], axis=1, inplace=True)
df[8]['Reaj.'] = df[8]['Reaj.'].apply(lambda x: x.split(" ")[0])
df[8]['Unnamed: 2'] = df[8]['Unnamed: 2'].apply(lambda x: x.split('C/Cop ')[1])
values = ['contrato', 'nome', 'dependencia', 'data_nascimento', 'data_inclusao', 'mensalidade', 'reajuste_retroativo']
keys = ['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 6','Reaj.']
d = dict(zip(keys, values))
df[8].rename(columns=d, inplace=True)
df[8]'''
#values = ['contrato', 'nome', 'dependencia', 'data_nascimento', 'data_inclusao', 'mensalidade', 'reajuste_retroativo']
#keys = ['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 6','Reaj.']
#d = dict(zip(keys, values))
#df[8].rename(columns=d, inplace=True)
#df[8].head()

In [ ]:
pages = df_filter_procedure(pages)
#pages = pd.concat([pages, df[8]])
pages.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)
pages.head()

In [ ]:
def convert_coin_(x):
    x = x.replace('.','')
    x = x.replace(',','.')
    return x
pages['mensalidade'] = pages['mensalidade'].apply(lambda x: convert_coin_(x))
#AttributeError: 'float' object has no attribute 'replace'

In [ ]:
pages['reajuste_retroativo'] = pages['reajuste_retroativo'].apply(lambda x: convert_coin_(x))
pages['reajuste_faixa'] = pages['reajuste_faixa'].apply(lambda x: convert_coin_(x))
#pages
#AttributeError: 'float' object has no attribute 'replace'

In [ ]:
pages

In [ ]:
titulares = []
titular_atual = None
pessoas_set = set()

for index, row in pages.iterrows():
    mensalidade = float(row[5])
    p = Person(row[1], mensalidade) 
    p.dependencia = row[2]
    p.data_nascimento = row[3]
    p.data_inclusao = row[4]
    p.reaj_retro = float(row[6])
    p.reaj_faixa = float(row[7])
    if p.dependencia == 'Titulares':
        titulares.append(p)
        titular_atual = p
    else:
        titular_atual.dependentes.append(p)
    pessoas_set.add(p)

In [ ]:
titulares

In [ ]:
#Extrai somente a página 1
area  = [93.426,99.023,818.448,343.567]
#ut = tabula.read_pdf(FILE_REPORT_USO, pages='all', lattice=True, multiple_tables=False)
ut = tabula.read_pdf(FILE_REPORT_USO, pages=1, stream=True, multiple_tables=True, area=area, pandas_options={'header':None})
ut[0]

In [ ]:
# segunda página
temp = pd.DataFrame([['Leticia Alves Fonseca', 'R$ 81,92'],
              ['Elza Moreira Alves', 'R$ 34,31'],
              ['Jefferson Castro Casseano Furtado', 'R$ 30,87'], 
              ['Aurea Dayse Cosmo Da Silva', 'R$ 103,81']])
ut[0] = pd.concat([ut[0], temp], ignore_index=True)
ut[0]

In [ ]:
#esta nao precisou em maio
#area_2 = [75.311,100.745,120.087,338.401]
#Extrai somente a página 2
#ut = tabula.read_pdf(FILE_REPORT_USO, pages='all', lattice=True, multiple_tables=False)
#ut2 = tabula.read_pdf(FILE_REPORT_USO, pages=2, stream=True, multiple_tables=False, area=area_2, pandas_options={'header':None})
#ut2[0]

In [ ]:
'''def replace_name(wrong_name, list_of_correct_names):
    for name in list_of_correct_names:
        if wrong_name in name:
            return name
        
nomes_corretos = [t.nome for t in titulares]
nomes_corretos.append("Ney Grequi Franco Figueiredo")
nomes_corretos.append("Alexandre Vieira Saboia")
nomes_corretos.append("Claudemir Miranda Barboza")'''

In [ ]:
ut0 = ut[0]
dict_ut = {}
for index, row in ut0.iterrows():
    #mensalidade = float(row[2][3:].replace(',','.'))
    #mensalidade = row[2]
    mensalidade = row[1]
    if mensalidade != 'Valor':
        mensalidade = float(row[1].replace(',','.').replace("R$ ",''))
        nome=row[0]
        dict_ut[nome] = mensalidade
        print(nome + " " + str(mensalidade))
    else:
        continue
sum(dict_ut.values())

In [ ]:
#não utilizada em maio
'''
ut2 = ut2[0]
dict_ut2 = {}
for index, row in ut2.iterrows():
    mensalidade = float(row[1][3:].replace(',','.'))
    #nome = replace_name(row[0], nomes_corretos)
    nome=row[0]
    dict_ut2[nome] = mensalidade
    print(nome)
sum(dict_ut2.values())'''

In [ ]:
#dict_ut2

In [ ]:
#dict_ut = {**dict_ut, **dict_ut2} #merge two dicts
checked = set()
#for titular in titulares:
#    if titular.nome in dict_ut:
#        titular.utilizacao = dict_ut[titular.nome]
#        checked.add(titular.nome)
for pessoa in pessoas_set:
    if pessoa.nome in dict_ut:
        pessoa.utilizacao = dict_ut[pessoa.nome]
        checked.add(pessoa.nome)

In [ ]:
# Encontra aqueles que não possuem mensalidade (desligados do plano), mas que 
# precisam pagar alguma utilização pendente
unchecked = set([t for t in dict_ut]) - checked
unchecked

In [ ]:
def computa_total_mensalidades(titulares):
    #return sum([t.get_valor_total() for t in titulares])
    return sum([t.get_valor_mensalidades() for t in titulares])

In [ ]:
def computa_total_utilizacao(titulares):
    return sum([t.utilizacao for t in titulares])

In [ ]:
def computa_reaj_retroativo(titulares):
    return sum([t.get_valor_reaj_retroativo() for t in titulares])

In [ ]:
def computa_reaj_faixa(titulares):
    return sum([t.get_valor_reaj_faixa() for t in titulares])

In [ ]:
computa_total_mensalidades(titulares)
#ok -> verificado

In [ ]:
computa_reaj_retroativo(titulares)

In [ ]:
computa_reaj_faixa(titulares)

In [ ]:
computa_total_utilizacao(titulares)
#erro: deveria ser 8.703,71

In [ ]:
def caso_especial_michel_osmar():
    for t in titulares:
        if t.nome == 'Michel Osmar Costa Paiva':
            mulher_do_michel = t.dependentes.pop(0)
            titulares.append(mulher_do_michel)
caso_especial_michel_osmar()

In [ ]:
computa_total_utilizacao(titulares)
#computa_total_mensalidades(titulares)

In [ ]:
from datetime import date
def generate_code(i):
    i = i + 1
    today = date.today()
    s = today.strftime("%d%m%Y")
    s = s + str(i)
    return s

In [ ]:
lista_final = []
for i,t in enumerate(titulares):
    v = f'{t.get_valor_total():.2f}'.replace('.',',')
    c = generate_code(i)
    l = [t.nome.upper(), v , c]
    lista_final.append(l)

In [ ]:
lista_final

In [ ]:
#lista_final.append(["CLAUDEMIR MIRANDA BARBOZA", '74,43', '1104202199'])
#94 de utilização e 3 do boleto

In [ ]:
#lista_final = lista_final[:-1]
#lista_final

In [ ]:
import json
with open('data-agosto-21.json', 'w') as f:
    json.dump(lista_final, f)

In [ ]:
titulares[0].report_values()

In [ ]:
len(titulares)

In [ ]:
len(lista_final)

In [ ]:
sorted(lista_final, key=lambda x: x[0])

In [ ]:
nomes = [t.nome.upper().replace(' ', '-') for t in titulares]

In [ ]:
#nomes

In [ ]:
import os

In [ ]:
#verifica se todos os boletos foram gerados
#arquivos = os.listdir('/home/juliano/Downloads/boletos-fev-21/')
#arquivos = [arq[:-4] for arq in arquivos if arq.endswith('.pdf')]
#set1 = set(arquivos)
#set2 = set(nomes)
#set2-set1

In [ ]:
for t in titulares:
    if t.nome.startswith('Laura'):
        print(t.report_values())

In [ ]:
import pickle
pickle.dump(titulares, open("titulares-agosto-21", "wb" ) )